# Imports

In [1]:
from enum import Enum
from os import listdir
import os.path

In [2]:
from tensorflow.keras.applications import vgg16
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

In [3]:
# Let keras load images that might be too small
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Constants

In [4]:
LOG_SEVERITY = 1

IMG_TGT_SZ = (224, 224)
IMGS_BASE_PATH = os.path.join('..', 'COVID-19 Dataset', 'CT', 'COVID')

# Support Functions

In [5]:
class Severity(Enum):
    DEBUG = 1
    INFO = 2
    WARNING = 3

def log_msg(msg, severity=Severity.INFO):
    if LOG_SEVERITY <= severity.value:
        print(f'{severity.name}: {msg}')

In [6]:
def preprocess_images(images, model_preprocess=None):
    "Preprocess a list of image files to be used with a CNN 224x224 model"
    pil_images = [load_img(image, target_size=IMG_TGT_SZ) for image in images]
    log_msg(f'Loaded {len(images)} images.')
    np_images = np.array([img_to_array(image) for image in pil_images])
    log_msg(f'Converted to numpy arrays with shape {np_images.shape}.')
    if model_preprocess:
        np_images = model_preprocess(np_images)
    return np_images

# Primary Execution flow

In [7]:
files = listdir(IMGS_BASE_PATH)

images = [os.path.join(IMGS_BASE_PATH, f) for f in files 
          if f.endswith(('png', 'PNG', 'jpg', 'JPG', 'jpeg', 'JPEG'))]
image_names = [os.path.split(f)[1].rpartition('.')[0] for f in images]

log_msg(f'Loading some images: {images[:5]}', Severity.DEBUG)
np_images = preprocess_images(images[:100], vgg16.preprocess_input)

DEBUG: Loading some images: ['../COVID-19 Dataset/CT/COVID/2020.03.01.20029769-p21-73_1%1.png', '../COVID-19 Dataset/CT/COVID/coronacases_org_001_z079.png', '../COVID-19 Dataset/CT/COVID/16745_2_2.PNG', '../COVID-19 Dataset/CT/COVID/coronacases_org_001_z045.png', '../COVID-19 Dataset/CT/COVID/2020.03.03.20030668-p18-64%1.png']
INFO: Loaded 100 images.
INFO: Converted to numpy arrays with shape (100, 224, 224, 3).


In [ ]:
model = vgg16.VGG16()
yhat = model.predict(np_images)
predictions = vgg16.decode_predictions(yhat)

In [ ]:
print('Top Predictions by vgg16')
print('-'*40)
for ndx, name in enumerate(image_names):
    print(f'{name}: {predictions[ndx][0][1]}')